In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt

In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import seaborn as sns
sns.set()

In [5]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

In [6]:
from sklearn.model_selection import KFold, StratifiedKFold

In [7]:
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv('test.csv', index_col=0)
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

In [8]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [9]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,8
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,8
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,8
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,8
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,10


In [10]:
train_X = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_X = test

In [11]:
train['type'].value_counts().sum()

199991

In [12]:
train['type'].value_counts()

QSO                    49680
GALAXY                 37347
SERENDIPITY_BLUE       21760
SPECTROPHOTO_STD       14630
REDDEN_STD             14618
STAR_RED_DWARF         13750
STAR_BHB               13500
SERENDIPITY_FIRST       7132
ROSAT_D                 6580
STAR_CATY_VAR           6506
SERENDIPITY_DISTANT     4654
STAR_CARBON             3257
SERENDIPITY_RED         2562
STAR_WHITE_DWARF        2160
STAR_SUB_DWARF          1154
STAR_BROWN_DWARF         500
SKY                      127
SERENDIPITY_MANUAL        61
STAR_PN                   13
Name: type, dtype: int64

In [13]:
#X_train, X_test, y_train, y_test = train_test_split(train_X, 
#                                                    train_y, 
#                                                    stratify = train_y,
#                                                    test_size=0.2, 
#                                                    random_state=1234)

### stratifiedKFold를 사용하자

In [14]:
#kfold = KFold(n_splits=6, random_state= 0, shuffle = True)
kfold_s = StratifiedKFold(n_splits=6, random_state= 0, shuffle = True)

# CatBoost

In [15]:
import catboost as catb
from catboost import CatBoostClassifier

In [24]:
CBC = CatBoostClassifier(random_state=1234)

cbc_param_grid = {
    #'n_estimators' : [10, 30, 50],
    #'max_depth' : np.arange(3,10,1),
    'learning_rate' : [0.1,0.15,0.01],#np.arange(0.159,0.18,0.01),
    'iterations' : [1000],
    'depth' : [8],
    'loss_function' : ['MultiClass'], #Logloss
    'leaf_estimation_method' : ['Newton'],
    'eval_metric' : ['Accuracy'], #AUC
}

In [25]:
n_iter_search = 10
CBC_rgs = RandomizedSearchCV(CBC, 
                             cbc_param_grid, 
                             cv=kfold_s,
                             scoring='neg_log_loss',
                             random_state=1234)

In [ ]:
CBC_rgc = CBC_rgs.fit(train_X, train_y)

C:\Users\1-18\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


0:	learn: 0.7145112	total: 345ms	remaining: 5m 44s
1:	learn: 0.7328369	total: 709ms	remaining: 5m 53s
2:	learn: 0.7372593	total: 1.09s	remaining: 6m 1s
3:	learn: 0.7514446	total: 1.47s	remaining: 6m 6s
4:	learn: 0.7559811	total: 1.83s	remaining: 6m 4s
5:	learn: 0.7636018	total: 2.2s	remaining: 6m 5s
6:	learn: 0.7708445	total: 2.58s	remaining: 6m 5s
7:	learn: 0.7717745	total: 2.93s	remaining: 6m 3s
8:	learn: 0.7730827	total: 3.27s	remaining: 6m
9:	learn: 0.7734547	total: 3.62s	remaining: 5m 58s
10:	learn: 0.7785972	total: 3.97s	remaining: 5m 56s
11:	learn: 0.7777391	total: 4.32s	remaining: 5m 55s
12:	learn: 0.7827436	total: 4.67s	remaining: 5m 54s
13:	learn: 0.7850418	total: 5.03s	remaining: 5m 53s
14:	learn: 0.7898302	total: 5.39s	remaining: 5m 53s
15:	learn: 0.7922185	total: 5.75s	remaining: 5m 53s
16:	learn: 0.7950867	total: 6.12s	remaining: 5m 53s
17:	learn: 0.7966829	total: 6.48s	remaining: 5m 53s
18:	learn: 0.7986331	total: 6.84s	remaining: 5m 52s
19:	learn: 0.8010033	total: 7.18s

In [19]:
CBC_rgc.best_estimator_.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'Accuracy',
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'approx_on_full_history': False,
 'use_best_model': False,
 'class_names': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18],
 'random_seed': 1234,
 'depth': 8,
 'has_time': False,
 'fold_len_multiplier': 2,
 'border_count': 254,
 'bagging_temperature': 1,
 'classes_count': 0,
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'loss_function': 'MultiClass',
 'learning_rate': 0.14900000393390656,
 'score_function': 'Cosine',


In [20]:
CBC_rgc.best_score_

-0.37368637329744214

In [21]:
CBC_best = CBC_rgc.best_estimator_
y_pred = CBC_best.predict_proba(test_X)

In [23]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission_alltest_200207.csv', index=True)